<a href="https://colab.research.google.com/github/LorenzoCorbinelli/MLSA-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount content/Drive

from google.colab import drive
drive.mount('/content/Drive')


Mounted at /content/Drive


In [ ]:
# prompt: parse a file inserting in a list the element of the file that are in between <CODESPLIT>

from google.colab import drive
drive.mount('/content/Drive')

def parse_file(filepath):
  with open(filepath, 'r') as file:
    content = file.read()
    # Split the content by <CODESPLIT>
    parts = content.split('<CODESPLIT>')

      # Remove the first element if it's empty (text before the first <CODESPLIT>)
    if parts[0] == '':
        parts.pop(0)

    # Filter out parts containing links
    filtered_parts = [part for part in parts if 'http://' not in part and 'https://' not in part]

    return filtered_parts

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
def parse_content(parts):
    parsed_data = []
    for i in range(len(parts)):
        if 'def ' in parts[i]:
            snippet_of_code = parts[i].strip()
            description = parts[i-1].strip() if i > 0 else "No description available"
            name_of_function = parts[i-2].strip() if i > 1 else "Unknown Function"

            parsed_data.append({
                "name of the function": name_of_function,
                "description": description,
                "snippet of code": snippet_of_code
            })

    return parsed_data

In [ ]:
import pandas as pd

# Example usage (assuming the file is in /content/Drive/My Drive/my_file.txt):
filepath = "/content/Drive/My Drive/MLSA/CodeSearch/train.txt"  # Replace with actual file path
extracted_code = parse_file(filepath)
parsed_data = parse_content(extracted_code)
df = pd.DataFrame(parsed_data)

In [ ]:
df.head(20)

,name of the function,description,snippet of code
0,InteractiveDecorator._logdef,Logs the definition of the object that was jus...,"def _logdef ( self , n , o , otype ) : import ..."
1,Writer._write_frames,Write metadata to a file handle .,"def _write_frames ( self , handle ) : assert h..."
2,WinkApiInterface.update_firmware,Make a call to the update_firmware endpoint . ...,"def update_firmware ( self , device , id_overr..."
3,select,datbase dialect,"def select ( table , key = 'default' ) : datab..."
4,Optimizer._rmsprop,Uses RMSProp to compute step from gradients .,"def _rmsprop ( self , grads , cache = None , d..."
5,ArcLine.linspace_pix,Return x y values evaluated with a given pixel...,"def linspace_pix ( self , start = None , stop ..."
6,BuildClient.save_template,SaveTemplate . Updates an existing build defin...,"def save_template ( self , template , project ..."
7,ScalableBloomFilter.fromfile,Serialize this ScalableBloomFilter into the fi...,"def fromfile ( cls , f ) : filter = cls ( ) fi..."
8,SamTemplate.get,Gets the resource at the given logicalId if pr...,"def get ( self , logicalId ) : if logicalId no..."
9,_resolve_items_to_service_uids,This function returns a list of UIDs from anal...,def _resolve_items_to_service_uids ( items ) :...


In [ ]:
print(df["description"].iloc[16])

Mainly used for Analysis Service s unit . Transform the text adding sub and super html scripts : For super - scripts use ^ char For sub - scripts use _ char The expression cm^2 will be translated to cm² and the expression b_ ( n - 1 ) will be translated to b n - 1 . The expression n_ ( fibras ) / cm^3 will be translated as n fibras / cm³ : param text : text to be formatted


In [ ]:
import tokenize
import token
import keyword
from io import StringIO
def categorize_token(tok):
    """
    Categorizes a token into specific types such as keyword, operator, identifier, etc.

    Args:
        tok (TokenInfo): A single token object from `tokenize`.

    Returns:
        dict: A dictionary containing token type and its categorization.
    """
    token_type = tok.type
    token_string = tok.string

    # Check if it's a Python keyword
    if token_type == token.NAME and keyword.iskeyword(token_string):
        category = 'Keyword'
    # Check if it's an operator
    elif token_type == token.OP:
        category = 'Operator'
    # Check if it's a literal (string, number, etc.)
    elif token_type in {token.STRING, token.NUMBER}:
        category = 'Literal'
    # Check if it's a name (identifier)
    elif token_type == token.NAME:
        category = 'Identifier'
    # Check if it's a punctuation (special character)
    elif token_type in {token.NEWLINE, token.INDENT, token.DEDENT, token.ENDMARKER}:
        category = 'Structure'
    # Fallback for other types
    else:
        category = tokenize.tok_name.get(token_type, 'Unknown')

    return {
        'type': token.tok_name[token_type],
        'string': token_string,
        'start': tok.start,
        'end': tok.end,
        'line': tok.line.strip(),
        'category': category
    }


In [ ]:
def tokenize_and_categorize(source_code):
    tokens = []
    reader = StringIO(source_code).readline
    try:
        for tok in tokenize.generate_tokens(reader):
            categorized = categorize_token(tok)
            tokens.append(categorized)
    except tokenize.TokenError as e:
        print(f"Tokenization error: {e}")
    return tokens

In [ ]:
tokens = tokenize_and_categorize(df["snippet of code"].iloc[16])

In [ ]:
def print_tokens(tokens):
    """
    Pretty prints categorized tokens.

    Args:
        tokens (list): List of categorized tokens.
    """
    print(f"{'Type':<15} {'String':<20} {'Category':<15} {'Start':<10} {'End':<10}")
    print("-" * 80)
    for tok in tokens:
        print(f"{tok['type']:<15} {tok['string']:<20} {tok['category']:<15} {str(tok['start']):<10} {str(tok['end']):<10}")


In [ ]:
print_tokens(tokens)

Type            String               Category        Start      End       
--------------------------------------------------------------------------------
NAME            def                  Keyword         (1, 0)     (1, 3)    
NAME            format_supsub        Identifier      (1, 4)     (1, 17)   
OP              (                    Operator        (1, 18)    (1, 19)   
NAME            text                 Identifier      (1, 20)    (1, 24)   
OP              )                    Operator        (1, 25)    (1, 26)   
OP              :                    Operator        (1, 27)    (1, 28)   
NAME            out                  Identifier      (1, 29)    (1, 32)   
OP              =                    Operator        (1, 33)    (1, 34)   
OP              [                    Operator        (1, 35)    (1, 36)   
OP              ]                    Operator        (1, 37)    (1, 38)   
NAME            subsup               Identifier      (1, 39)    (1, 45)   
OP              =  

In [ ]:
import re
def custom_tokenizer(code):
    # Define regex patterns for different token types
    token_specification = [
        ('NUMBER',    r'\b\d+(\.\d*)?\b'),  # Integer or decimal number
        ('ASSIGN',    r'='),                # Assignment operator
        ('END',       r';'),                # Statement terminator
        ('ID',        r'\b[A-Za-z_]\w*\b'), # Identifiers
        ('OP',        r'[+\-*/]'),          # Arithmetic operators
        ('NEWLINE',   r'\n'),               # Line endings
        ('PAREN',     r'[({}\[\])]'),       # Parentheses
        ('SKIP',      r'[ \t]+'),           # Skip over spaces and tabs
        ('MISMATCH',  r'.'),                # Any other character
    ]
    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    get_token = re.compile(tok_regex).finditer
    tokens = []
    for match in get_token(code):
        kind = match.lastgroup
        value = match.group()
        if kind == 'ID' and value in {'def', 'return', 'if', 'else', 'for', 'while'}:
            kind = 'KEYWORD'
        elif kind == 'SKIP':
            continue
        elif kind == 'MISMATCH':
            continue  # Ignore mismatched characters instead of raising an error
        tokens.append((kind, value))
    return tokens

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

# length of df
df.describe()


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,name of the function,description,snippet of code
count,810928,810928,810928
unique,362847,377493,557399
top,main,Auto Generated Code,Dump the function def of a macro or call block .
freq,3053,3251,6


In [ ]:
print(df['snippet of code'].iloc[0])

def _logdef ( self , n , o , otype ) : import re try : #The latest input cell will be the one that this got executed #from. TODO: actually, if acorn got imported after the fact, then #the import would have caused all the undecorated functions to be #decorated as soon as acorn imported. I suppose we just won't have #any code for that case. if otype == "classes" : cellno = max ( [ int ( k [ 2 : ] ) for k in self . shell . user_ns . keys ( ) if re . match ( "_i\d+" , k ) ] ) elif otype == "functions" : cellno = int ( o . __code__ . co_filename . strip ( "<>" ) . split ( '-' ) [ 2 ] ) except : #This must not have been an ipython notebook declaration, so we #don't store the code. cellno = None pass code = "" if cellno is not None : cellstr = "_i{0:d}" . format ( cellno ) if cellstr in self . shell . user_ns : cellcode = self . shell . user_ns [ cellstr ] import ast astm = ast . parse ( cellcode ) ab = astm . body parts = { ab [ i ] . name : ( ab [ i ] . lineno , None if i + 1 >= len ( ab ) 

In [ ]:
# Tokenize only the first element of df["snippet of code"]
first_snippet_tokenized = custom_tokenizer(df['snippet of code'].iloc[0])

# Print the tokenized first snippet of code
print(first_snippet_tokenized)

[('KEYWORD', 'def'), ('ID', '_logdef'), ('PAREN', '('), ('ID', 'self'), ('ID', 'n'), ('ID', 'o'), ('ID', 'otype'), ('PAREN', ')'), ('ID', 'import'), ('ID', 're'), ('ID', 'try'), ('ID', 'The'), ('ID', 'latest'), ('ID', 'input'), ('ID', 'cell'), ('ID', 'will'), ('ID', 'be'), ('ID', 'the'), ('ID', 'one'), ('ID', 'that'), ('ID', 'this'), ('ID', 'got'), ('ID', 'executed'), ('ID', 'from'), ('ID', 'TODO'), ('ID', 'actually'), ('KEYWORD', 'if'), ('ID', 'acorn'), ('ID', 'got'), ('ID', 'imported'), ('ID', 'after'), ('ID', 'the'), ('ID', 'fact'), ('ID', 'then'), ('ID', 'the'), ('ID', 'import'), ('ID', 'would'), ('ID', 'have'), ('ID', 'caused'), ('ID', 'all'), ('ID', 'the'), ('ID', 'undecorated'), ('ID', 'functions'), ('ID', 'to'), ('ID', 'be'), ('ID', 'decorated'), ('ID', 'as'), ('ID', 'soon'), ('ID', 'as'), ('ID', 'acorn'), ('ID', 'imported'), ('ID', 'I'), ('ID', 'suppose'), ('ID', 'we'), ('ID', 'just'), ('ID', 'won'), ('ID', 't'), ('ID', 'have'), ('ID', 'any'), ('ID', 'code'), ('KEYWORD', 'for'

In [ ]:
def process_in_chunks(df, chunk_size=100):
    """
    Processes the DataFrame in consecutive chunks, applying custom_tokenizer.

    Args:
        df: The pandas DataFrame to process.
        chunk_size: The number of rows to process in each chunk.

    Returns:
        The DataFrame with the 'tokenized_snippet' column added.
    """
    df['tokenized_snippet'] = ''  # Initialize the column
    num_chunks = (len(df) + chunk_size - 1) // chunk_size  # Calculate the number of chunks

    for i in range(num_chunks):
        start_index = i * chunk_size
        end_index = min((i + 1) * chunk_size, len(df))  # Ensure we don't go beyond the DataFrame bounds
        chunk = df['snippet of code'].iloc[start_index:end_index]
        df['tokenized_snippet'].iloc[start_index:end_index] = chunk.apply(custom_tokenizer)

    return df

In [ ]:
df_tokenized = process_in_chunks(df, chunk_size=100)
df_tokenized.head()

Output streaming troncato alle ultime 5000 righe.

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['tokenized_snippet'].iloc[start_index:end_index] = chunk.apply(custom_tokenizer)
<ipython-input-11-0ebe5d4bd030>:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer,

KeyboardInterrupt: 

In [ ]:
df['tokenized_snippet'].iloc[0]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Step 1: Download the dataset
!wget http://files.srl.inf.ethz.ch/data/py150_files.tar.gz

Mounted at /content/drive
--2024-12-20 14:26:09--  http://files.srl.inf.ethz.ch/data/py150_files.tar.gz
Resolving files.srl.inf.ethz.ch (files.srl.inf.ethz.ch)... 129.132.114.90
Connecting to files.srl.inf.ethz.ch (files.srl.inf.ethz.ch)|129.132.114.90|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.sri.inf.ethz.ch/data/py150_files.tar.gz [following]
--2024-12-20 14:26:09--  https://files.sri.inf.ethz.ch/data/py150_files.tar.gz
Resolving files.sri.inf.ethz.ch (files.sri.inf.ethz.ch)... 129.132.114.90
Connecting to files.sri.inf.ethz.ch (files.sri.inf.ethz.ch)|129.132.114.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199067128 (190M) [application/gzip]
Saving to: ‘py150_files.tar.gz’

py150_files.tar.gz  100%[===================>] 189.84M  22.9MB/s    in 9.7s    

2024-12-20 14:26:20 (19.6 MB/s) - ‘py150_files.tar.gz’ saved [199067128/199067128]



In [2]:
!tar -xvzf py150_files.tar.gz

data.tar.gz
github_repos.txt
python100k_train.txt
python50k_eval.txt
README.md


In [3]:
!tar -xvzf data.tar.gz

Output streaming troncato alle ultime 5000 righe.
data/mathcamp/pypicloud/git_hooks/hook.py
data/mathcamp/pypicloud/tests/
data/mathcamp/pypicloud/tests/test_simple.py
data/mathcamp/pypicloud/tests/test_packages.py
data/mathcamp/pypicloud/tests/test_access_backends.py
data/mathcamp/pypicloud/tests/test_security.py
data/mathcamp/pypicloud/tests/test_api.py
data/mathcamp/pypicloud/tests/test_storage.py
data/mathcamp/pypicloud/tests/test_util.py
data/mathcamp/pypicloud/tests/test_auth.py
data/mathcamp/pypicloud/pypicloud/
data/mathcamp/pypicloud/pypicloud/scripts.py
data/mathcamp/pypicloud/pypicloud/storage/
data/mathcamp/pypicloud/pypicloud/storage/files.py
data/mathcamp/pypicloud/pypicloud/views/
data/mathcamp/pypicloud/pypicloud/views/packages.py
data/mathcamp/pypicloud/pypicloud/views/simple.py
data/mathcamp/pypicloud/pypicloud/views/api.py
data/mathcamp/pypicloud/pypicloud/models.py
data/mathcamp/pypicloud/pypicloud/cache/
data/mathcamp/pypicloud/pypicloud/cache/__init__.py
data/math

In [ ]:
!head -n 5 python100k_train.json > subset.json

In [ ]:
!split -l 1000 python100k_train.json chunk_ # split the json into chunck files

In [ ]:
# prompt: open the file subset.json and load data --> continue also if not well formatted

import json
import os

def load_json_data(filepath):
    data = []
    try:
        with open(filepath, 'r') as f:
            for line in f:
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Skipping invalid JSON line: {line.strip()}")
                    print(f"Error: {e}")
    except FileNotFoundError:
        print(f"File not found: {filepath}")
    return data

# Example usage:
filepath = "subset.json"
loaded_data = load_json_data(filepath)

# Now you can work with the 'loaded_data' which is a list of dictionaries
# Example: Print the first few items
if loaded_data:
    for i in loaded_data[:5]:
        print(i)

[{'type': 'Module', 'children': [1, 3, 5, 7, 9, 11]}, {'type': 'Expr', 'children': [2]}, {'type': 'Str', 'value': ' Provides ``mapping`` of url paths to request handlers.\n'}, {'type': 'ImportFrom', 'children': [4], 'value': 'bootstrap'}, {'type': 'alias', 'value': 'Bootstrap'}, {'type': 'ImportFrom', 'children': [6], 'value': 'fund'}, {'type': 'alias', 'value': 'InstantPaymentNotificationHandler'}, {'type': 'ImportFrom', 'children': [8], 'value': 'fund'}, {'type': 'alias', 'value': 'ThankYouHandler'}, {'type': 'ImportFrom', 'children': [10], 'value': 'view'}, {'type': 'alias', 'value': '*'}, {'type': 'Assign', 'children': [12, 13]}, {'type': 'NameStore', 'value': 'mapping'}, {'type': 'ListLoad', 'children': [14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98, 101, 104, 107, 110, 113]}, {'type': 'TupleLoad', 'children': [15, 16]}, {'type': 'Str', 'value': '/'}, {'type': 'NameLoad', 'value': 'Index'}, {'type': 'TupleLoad', 

In [ ]:
def split_large_json(input_file, lines_per_file):
    with open(input_file, 'r') as f:
        for i in range(4):  # Since you want 4 files
            with open(f'sub_file_{(i+1)*10}k.json', 'w') as out_file:
                for _ in range(lines_per_file):
                    line = f.readline()
                    if not line:
                        break
                    out_file.write(line)

# Usage
split_large_json('python100k_train.json', 10000)

In [ ]:
# prompt: load each sub_file_25k.json, 50k.json, and so on (until 100k) into different dataframe and then merge them

import pandas as pd
import json

def load_json_to_dataframe(filepath):
    try:
        with open(filepath, 'r') as f:
            data = [json.loads(line) for line in f]
        return pd.DataFrame(data)
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in '{filepath}': {e}")
        return None


In [ ]:
filename = f"sub_file_10k.json"
df = load_json_to_dataframe(filename)
df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,26487,26488,26489,26490,26491,26492,26493,26494,26495,26496
0,"{'type': 'Module', 'children': [1, 3, 5, 7, 9,...","{'type': 'Expr', 'children': [2]}","{'type': 'Str', 'value': ' Provides ``mapping`...","{'type': 'ImportFrom', 'children': [4], 'value...","{'type': 'alias', 'value': 'Bootstrap'}","{'type': 'ImportFrom', 'children': [6], 'value...","{'type': 'alias', 'value': 'InstantPaymentNoti...","{'type': 'ImportFrom', 'children': [8], 'value...","{'type': 'alias', 'value': 'ThankYouHandler'}","{'type': 'ImportFrom', 'children': [10], 'valu...",...,None,None,None,None,None,None,None,None,None,None
1,"{'type': 'Module', 'children': [1, 3, 5, 7, 9,...","{'type': 'Import', 'children': [2]}","{'type': 'alias', 'value': 'msgpack'}","{'type': 'Import', 'children': [4]}","{'type': 'alias', 'value': 'gevent.pool'}","{'type': 'Import', 'children': [6]}","{'type': 'alias', 'value': 'gevent.queue'}","{'type': 'Import', 'children': [8]}","{'type': 'alias', 'value': 'gevent.event'}","{'type': 'Import', 'children': [10]}",...,None,None,None,None,None,None,None,None,None,None
2,"{'type': 'Module', 'children': [1, 3, 5, 7]}","{'type': 'Expr', 'children': [2]}","{'type': 'Str', 'value': 'Django's command lin...","{'type': 'Import', 'children': [4]}","{'type': 'alias', 'value': 'os'}","{'type': 'Import', 'children': [6]}","{'type': 'alias', 'value': 'sys'}","{'type': 'If', 'children': [8, 11]}","{'type': 'CompareEq', 'children': [9, 10]}","{'type': 'NameLoad', 'value': '__name__'}",...,None,None,None,None,None,None,None,None,None,None
3,"{'type': 'Module', 'children': [1, 3, 5, 14, 3...","{'type': 'Expr', 'children': [2]}","{'type': 'Str', 'value': 'Installer for hippyb...","{'type': 'Import', 'children': [4]}","{'type': 'alias', 'value': 'os'}","{'type': 'Assign', 'children': [6, 7]}","{'type': 'NameStore', 'value': 'cwd'}","{'type': 'Call', 'children': [8, 13]}","{'type': 'AttributeLoad', 'children': [9, 12]}","{'type': 'AttributeLoad', 'children': [10, 11]}",...,None,None,None,None,None,None,None,None,None,None
4,"{'type': 'Module', 'children': [1, 3, 5]}","{'type': 'Import', 'children': [2]}","{'type': 'alias', 'value': 'os'}","{'type': 'Import', 'children': [4]}","{'type': 'alias', 'value': 'sys'}","{'type': 'If', 'children': [6, 9]}","{'type': 'CompareEq', 'children': [7, 8]}","{'type': 'NameLoad', 'value': '__name__'}","{'type': 'Str', 'value': '__main__'}","{'type': 'body', 'children': [10, 19, 21]}",...,None,None,None,None,None,None,None,None,None,None
5,"{'type': 'Module', 'children': [1, 3, 6]}","{'type': 'ImportFrom', 'children': [2], 'value...","{'type': 'alias', 'value': 'unicode_literals'}","{'type': 'ImportFrom', 'children': [4, 5], 'va...","{'type': 'alias', 'value': 'models'}","{'type': 'alias', 'value': 'migrations'}","{'type': 'ClassDef', 'children': [7, 11, 76], ...","{'type': 'bases', 'children': [8]}","{'type': 'AttributeLoad', 'children': [9, 10]}","{'type': 'NameLoad', 'value': 'migrations'}",...,None,None,None,None,None,None,None,None,None,None
6,"{'type': 'Module', 'children': [1, 3, 5, 7, 9,...","{'type': 'Import', 'children': [2]}","{'type': 'alias', 'value': 'twitter'}","{'type': 'ImportFrom', 'children': [4], 'value...","{'type': 'alias', 'value': 'messages'}","{'type': 'ImportFrom', 'children': [6], 'value...","{'type': 'alias', 'value': 'user_passes_test'}","{'type': 'ImportFrom', 'children': [8], 'value...","{'type': 'alias', 'value': 'transaction'}","{'type': 'ImportFrom', 'children': [10, 11], '...",...,None,None,None,None,None,None,None,None,None,None
7,"{'type': 'Module', 'children': [1, 3, 8, 16]}","{'type': 'Expr', 'children': [2]}","{'type': 'Str', 'value': ' Exceptions raised b...","{'type': 'Assign', 'children': [4, 5]}","{'type': 'NameStore', 'value': '__all__'}","{'type': 'TupleLoad', 'children': [6, 7]}","{'type': 'Str', 'value': 'DjangoWSGIException'}","{'type': 'Str', 'value': 'ApplicationCallError'}","{'type': 'ClassDef', 'children': [9, 11, 15], ...","{'type': 'bases', 

In [ ]:
!pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.9 MB/s eta 0:00:00


In [ ]:
def preprocess_json(input_file, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    with open(output_file, 'w') as f:
        f.write('[')
        for line in lines:
            f.write(line.strip())
            if not line.strip().endswith(','):
                f.write(',')
        f.write(']')

# Usage
preprocess_json('python100k_train.json', 'cleaned_file.json')

In [ ]:
import ijson
import json

def split_large_json(input_file, lines_per_file):
    with open(input_file, 'r') as f:
        parser = ijson.items(f, 'item')
        sub_data = []
        file_index = 1
        line_count = 0

        for item in parser:
            sub_data.append(item)
            line_count += 1

            if line_count >= lines_per_file:
                with open(f'sub_file_{file_index}.json', 'w') as out_file:
                    json.dump(sub_data, out_file, indent=4)
                sub_data = []
                file_index += 1
                line_count = 0

        # Write remaining data if any
        if sub_data:
            with open(f'sub_file_{file_index}.json', 'w') as out_file:
                json.dump(sub_data, out_file, indent=4)

# Usage
split_large_json('cleaned_file.json', 25000)

After unzipping the data.tar.gz folder, we retrieve the data folder with all the snippets of code that will end in .py

In [24]:
# prompt: load all the python file recursive in the data folder in a dataframe (one for each row)

import os
import pandas as pd

def load_python_files(root_dir):
    """Loads all Python files in a directory recursively into a DataFrame."""
    all_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".py"):
                filepath = os.path.join(dirpath, filename)
                try:
                    with open(filepath, 'r') as f:  # Specify UTF-8 encoding
                        content = f.read()
                        all_files.append({'filepath': filepath, 'snippet_of_code': content})
                except Exception as e:
                    print(f"Error reading file {filepath}: {e}")
    return pd.DataFrame(all_files)


Extract all the source code of the python files into a dataframe. One file is reported into a single line, including the reference of the file itself

In [25]:
# Example usage:
data_dir = "/content/data"  # Replace with the actual path to your data folder
df_python_files = load_python_files(data_dir)

Error reading file /content/data/jpm/papercut/storage/forwarding_proxy.py: 'utf-8' codec can't decode byte 0xe4 in position 428: invalid continuation byte
Error reading file /content/data/CouchPotato/CouchPotatoServer/libs/xmpp/session.py: 'utf-8' codec can't decode byte 0xe4 in position 7553: invalid continuation byte
Error reading file /content/data/cloudera/hue/desktop/core/ext-py/tablib-0.10.0/tablib/packages/xlrd/formula.py: 'utf-8' codec can't decode byte 0xa9 in position 104: invalid start byte
Error reading file /content/data/cloudera/hue/desktop/core/ext-py/tablib-0.10.0/tablib/packages/xlrd/formatting.py: 'utf-8' codec can't decode byte 0xa9 in position 82: invalid start byte
Error reading file /content/data/cloudera/hue/desktop/core/ext-py/tablib-0.10.0/tablib/packages/xlrd/biffh.py: 'utf-8' codec can't decode byte 0xa9 in position 94: invalid start byte
Error reading file /content/data/cloudera/hue/desktop/core/ext-py/pysqlite/pysqlite2/test/userfunctions.py: 'utf-8' codec 

Checkout the numbers of files loaded

In [34]:
df_python_files["snippet_of_code"].count()

149972

Checkout which is the first file in the df

In [43]:
df_python_files.iloc[0]

,0
filepath,/content/data/tomcz/aws_py/s3client_tests.py
snippet_of_code,"from s3client import Parameters, S3Client, Bad..."


Checkout the content of this file, comparing it with the actual one, using cat

In [42]:
df_python_files["snippet_of_code"].iloc[0]

'from s3client import Parameters, S3Client, BadHttpResponse\nimport unittest, os, tempfile, datetime, urllib, uuid\nfrom properties import Credentials, loadcredentials\n\nclass CredentialsTests(unittest.TestCase):\n    def testCreatesExpectedSignature(self):\n        credentials = Credentials("test", "uV3F3YluFJax1cknvbcGwgjvx4QpvB+leU8dUj2o")\n        textToSign = "GET\\n\\n\\nTue, 27 Mar 2007 19:36:42 +0000\\n/johnsmith/photos/puppy.jpg"\n        self.assertEqual("xXjDGYUmKxnwqr5KXNPGldn5LbA=", credentials.sign(textToSign))\n\nclass ParametersTests(unittest.TestCase):\n    def testCreatesExpectedStringForObjectGet(self):\n        headers = {"Date": "Tue, 27 Mar 2007 19:36:42 +0000"}\n        params = Parameters(\'GET\', \'johnsmith\', \'puppy.jpg\', headers)\n        expected = "GET\\n\\n\\nTue, 27 Mar 2007 19:36:42 +0000\\n/johnsmith/puppy.jpg"\n        self.assertEqual(expected, params.textToSign())\n\n    def testCreatesExpectedStringForObjectGetWithPath(self):\n        headers = 

In [44]:
!cat /content/data/tomcz/aws_py/s3client_tests.py

from s3client import Parameters, S3Client, BadHttpResponse
import unittest, os, tempfile, datetime, urllib, uuid
from properties import Credentials, loadcredentials

class CredentialsTests(unittest.TestCase):
    def testCreatesExpectedSignature(self):
        credentials = Credentials("test", "uV3F3YluFJax1cknvbcGwgjvx4QpvB+leU8dUj2o")
        textToSign = "GET\n\n\nTue, 27 Mar 2007 19:36:42 +0000\n/johnsmith/photos/puppy.jpg"
        self.assertEqual("xXjDGYUmKxnwqr5KXNPGldn5LbA=", credentials.sign(textToSign))

class ParametersTests(unittest.TestCase):
    def testCreatesExpectedStringForObjectGet(self):
        headers = {"Date": "Tue, 27 Mar 2007 19:36:42 +0000"}
        params = Parameters('GET', 'johnsmith', 'puppy.jpg', headers)
        expected = "GET\n\n\nTue, 27 Mar 2007 19:36:42 +0000\n/johnsmith/puppy.jpg"
        self.assertEqual(expected, params.textToSign())

    def testCreatesExpectedStringForObjectGetWithPath(self):
        headers = {"Date": "Tue, 27 Mar 2007 19:36:

<h1>Tokenization</h1>
Now we will try to properly tokenize the pythons files.

In [48]:
type(df_python_files["snippet_of_code"].iloc[0])

str

In [47]:
# prompt: use built in function of python to tokenize the python code that is passed through dataframe

def tokenize_python_code(code):

    try:
        tokens = list(tokenize.tokenize(StringIO(code).readline))
        return tokens
    except tokenize.TokenError as e:
        print(f"Tokenization error: {e}")
        return []

# Apply the function to the 'snippet_of_code' column
df_python_files['tokenized_code'] = df_python_files['snippet_of_code'].apply(tokenize_python_code)

# Display the first few rows with tokenized code
df_python_files.head()

IndentationError: unexpected indent (<ipython-input-47-ba04f771154e>, line 5)